# FGM 로그 분석기 (Scoring Edition)

시간(Time) 대신 **주행 안정성(Stability)**과 **안전(Safety)**을 기준으로 점수를 매겨 최적의 파라미터를 찾습니다.

In [ ]:
import os
import re
import math
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 스타일 설정
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# -----------------------
# [설정] 분석 및 점수 가중치 파라미터
# -----------------------
class Config:
    LOG_DIR = "./logs/fgm"          # 로그 파일 경로
    MIN_LAP_TIME = 20.0             # 최소 주행 시간
    GOAL_RADIUS = 1.0               # 목표 도달 반경
    
    # [점수 산정 가중치]
    # 점수 = 100 - (W_LAT * avg_a_lat) - (W_YAW * avg_yaw) + (W_DIST * min_dist) - (W_COLL * collision)
    W_LAT_ACCEL = 20.0      # 횡가속도(쏠림) 페널티 가중치 (클수록 부드러운 주행 선호)
    W_YAW_RATE = 10.0       # 회전각속도(흔들림) 페널티 가중치
    W_OBS_DIST = 5.0        # 장애물 거리 보너스 (멀리서 피할수록 좋음)
    W_COLLISION = 100.0     # 충돌 시 차감 점수
    
    # 분석할 맵 리스트
    MAPS = ["Opp_bumper_v_2", "Opp_bumper_slow_1", "Opp_bumper_slow_2"] 

config = Config()

# 로그 디렉토리 확인
if not os.path.exists(config.LOG_DIR):
    print(f"알림: '{config.LOG_DIR}' 폴더를 생성했습니다. CSV 파일들을 넣어주세요.")
    os.makedirs(config.LOG_DIR, exist_ok=True)


In [ ]:
# =========================================================
# 1) 파일명 파싱
# =========================================================
FNAME_PATTERN = re.compile(
    r'(?P<map>.+)\.csv_Gap(?P<gap>[\d.]+)_Bub(?P<bub>[\d.]+)_Clr(?P<clr>[\d.]+)_WW(?P<ww>[\d.]+)_AW(?P<aw>[\d.]+)_SW(?P<sw>[\d.]+)_HB(?P<hb>[\d.]+)_CT(?P<ct>[\d.]+)_SA(?P<sa>[\d.]+)_DBS(?P<dbs>[\d.]+)_SPD(?P<spd>[\d.]+)_FGM_(?P<id>\d+)_(?P<result>[A-Z_]+)_dur_(?P<dur>[\d.]+)s\.csv'
)

def parse_filename(fname: str):
    m = FNAME_PATTERN.match(fname)
    if not m:
        return None
    d = m.groupdict()
    # 파라미터 실수형 변환
    float_keys = ["gap", "bub", "clr", "ww", "aw", "sw", "hb", "ct", "sa", "dbs", "spd", "dur"]
    for k in float_keys:
        d[k] = float(d[k])
    d['id'] = int(d['id'])
    return d

# =========================================================
# 2) 수치 데이터 및 주행 지표 추출
# =========================================================
def _is_float_str(s):
    try:
        float(s)
        return True
    except:
        return False

def extract_metrics(df: pd.DataFrame):
    # 유효한 수치 행만 필터링
    mask = df["t"].apply(lambda v: isinstance(v, (int, float)) or (isinstance(v, str) and _is_float_str(v)))
    num = df[mask].copy()
    
    if num.empty:
        return None

    # 주요 컬럼 실수형 변환
    target_cols = ["t", "x", "y", "a_lat", "yaw_rate", "min_d", "collision"]
    for col in target_cols:
        if col in num.columns:
            num[col] = pd.to_numeric(num[col], errors='coerce')
    
    # [핵심] 주행 안정성 지표 계산
    # 1. 평균 횡가속도 (절대값): 낮을수록 부드러운 주행 (Jerky 방지)
    avg_a_lat = num['a_lat'].abs().mean() if 'a_lat' in num.columns else 0.0
    max_a_lat = num['a_lat'].abs().max() if 'a_lat' in num.columns else 0.0
    
    # 2. 평균 회전각속도 (절대값): 낮을수록 안정적
    avg_yaw_rate = num['yaw_rate'].abs().mean() if 'yaw_rate' in num.columns else 0.0
    
    # 3. 장애물 최소 거리: 클수록 안전
    # (참고: 로그에서 min_d가 0인 경우 센서 미감지일 수 있음)
    min_obs_dist = num['min_d'].max() if 'min_d' in num.columns else 0.0
    
    return {
        "n_rows": len(num),
        "last_time": num["t"].iloc[-1],
        "avg_a_lat": avg_a_lat,
        "max_a_lat": max_a_lat,
        "avg_yaw_rate": avg_yaw_rate,
        "min_obs_dist": min_obs_dist
    }

# =========================================================
# 3) 개별 분석 실행
# =========================================================
def analyze_run(path: str) -> dict:
    fname = os.path.basename(path)
    meta = parse_filename(fname)
    if meta is None:
        raise ValueError(f"파일명 형식 불일치: {fname}")

    df = pd.read_csv(path)
    metrics = extract_metrics(df)
    
    # 결과 정보
    result = meta["result"]
    
    # 데이터가 없거나 로딩 실패 시 기본값
    if metrics is None:
        return {**meta, "score": 0, "note": "No Data"}

    # [점수 계산 로직]
    # 기본 점수 100점 시작
    score = 100.0
    
    # 1. 안정성 페널티 (Jerky)
    score -= (metrics['avg_a_lat'] * config.W_LAT_ACCEL)
    score -= (metrics['avg_yaw_rate'] * config.W_YAW_RATE)
    
    # 2. 장애물 거리 보너스 (Safety)
    score += (metrics['min_obs_dist'] * config.W_OBS_DIST)
    
    # 3. 결과 페널티
    is_collision = (result == "CRASH") or (df['collision'] == 1).any() if 'collision' in df.columns else False
    if is_collision:
        score -= config.W_COLLISION
        result = "CRASH" # 덮어쓰기
    elif result != "GOAL":
        score -= 50.0 # 완주 못함 페널티
        
    return {
        **meta,
        **metrics,
        "final_result": result,
        "total_score": round(score, 2),
        "is_collision": is_collision,
        "file_path": path
    }

# =========================================================
# 4) 최신 파일 선별
# =========================================================
def get_latest_log_files(log_dir):
    all_files = glob.glob(os.path.join(log_dir, "*.csv"))
    latest_files_map = {}
    
    for fpath in all_files:
        fname = os.path.basename(fpath)
        meta = parse_filename(fname)
        if meta is None: continue
            
        key = (
            meta['map'], meta['gap'], meta['bub'], meta['clr'], meta['ww'], 
            meta['aw'], meta['sw'], meta['hb'], meta['ct'], meta['sa'], 
            meta['dbs'], meta['spd']
        )
        if key not in latest_files_map or meta['id'] > latest_files_map[key][0]:
            latest_files_map[key] = (meta['id'], fpath)
            
    return [v[1] for v in latest_files_map.values()]


In [ ]:
target_files = get_latest_log_files(config.LOG_DIR)
results_list = []

for fpath in target_files:
    try:
        info = analyze_run(fpath)
        results_list.append(info)
    except Exception as e:
        print(f"Error: {e}")

df_results = pd.DataFrame(results_list)

if not df_results.empty:
    print(f"\n총 {len(df_results)}개의 시뮬레이션 분석 완료")
    
    # 점수가 높은 순서대로 정렬 (Best Case)
    df_results = df_results.sort_values(by='total_score', ascending=False)
    
    # 출력 컬럼 선택
    cols = ['map', 'total_score', 'final_result', 'avg_a_lat', 'avg_yaw_rate', 'min_obs_dist'] +            ['gap', 'bub', 'clr', 'ww'] # 주요 파라미터 일부만 표시
    
    print("\n==== [Top 5] 종합 점수(안정성+안전) 기준 베스트 ====")
    display(df_results[cols].head(5))
    
    print("\n==== [Worst 5] 점수 하위 (불안정/충돌) ====")
    display(df_results[cols].tail(5))
else:
    print("분석할 데이터가 없습니다.")


In [ ]:
# 맵별로 가장 점수가 높은 파라미터 조합 찾기
if not df_results.empty:
    best_per_map = df_results.loc[df_results.groupby("map")["total_score"].idxmax()]
    
    print("\n==== 각 맵별 최고의 안정성(Score)을 보인 파라미터 ====")
    display(best_per_map[['map', 'total_score', 'avg_a_lat', 'gap', 'bub', 'clr', 'ww', 'aw', 'sw']])
    
    # 인사이트 시각화
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df_results, x='avg_a_lat', y='total_score', hue='final_result', style='map', s=100)
    plt.title("Lateral Acceleration (Jerky) vs Total Score")
    plt.xlabel("Average Lateral Accel (Lower is smoother)")
    plt.ylabel("Total Score")
    plt.axvline(x=0.5, color='r', linestyle='--', alpha=0.3, label='Stability Threshold')
    plt.legend()
    plt.show()
